In [5]:
import s3fs
import pandas as pd

In [6]:
# Information d'accès au cloud MinIO (Bucket de William)
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "williamolivier"

In [ ]:
# Vérification du contenu
fs.ls(f"{MY_BUCKET}/diffusion")

In [ ]:
# Récupération des tableaux
FILE_PATH_S3_DCB = f"{MY_BUCKET}/diffusion/df_dossier_complet_brut.csv"
with fs.open(FILE_PATH_S3_DCB, "rb") as file:
    df_dossier_complet_brut = pd.read_csv(file)

# Récupération des tableaux
FILE_PATH_S3_MDCB = f"{MY_BUCKET}/diffusion/df_meta_dossier_complet_brut.csv"
with fs.open(FILE_PATH_S3_MDCB, "rb") as file:
    df_meta_dossier_complet_brut = pd.read_csv(file)

# Récupération des tableaux
FILE_PATH_S3_MB = f"{MY_BUCKET}/diffusion/df_mobpro_brut.csv"
with fs.open(FILE_PATH_S3_MB, "rb") as file:
    df_mobpro_brut = pd.read_csv(file)

# Récupération des tableaux
FILE_PATH_S3_MMB = f"{MY_BUCKET}/diffusion/df_meta_mobpro_brut.csv"
with fs.open(FILE_PATH_S3_MMB, "rb") as file:
    df_meta_mobpro_brut = pd.read_csv(file)

In [ ]:
df_mobpro_brut.sample()

In [65]:
# Homogénéisation des codes insee (tous en chaîne de caractère)
df_mobpro_brut["COMMUNE"] = df_mobpro_brut["COMMUNE"].astype(str).str.zfill(5)
df_mobpro_brut["DCLT"] = df_mobpro_brut["DCLT"].astype(str).str.zfill(5)
df_mobpro_brut["ARM"] = df_mobpro_brut["ARM"].astype(str).str.zfill(5)

La fonction ci-dessous permet d'obtenir un premier tableau regroupant ceratines informations par commune :
- Nombre de personnes du ménage (regroupé)
- Nombre de personnes scolarisées du ménage
- Nombre de personnes actives ayant un emploi du ménage
- Nombre d'élèves, étudiants ou stagiaires âgés de 14 ans ou plus du ménage
- Indicateur du lieu de travail
(1 Dans la commune de résidence actuelle
2 Dans une autre commune du département de résidence
3 Dans un autre département de la région de résidence
4 Hors de la région de résidence actuelle : en métropole
5 Hors de la région de résidence actuelle : dans un DOM
6 Hors de la région de résidence actuelle : dans une COM
7 A l'étranger)

In [11]:
# Conversion des colonnes en numérique
df_mobpro_brut['NPERR'] = pd.to_numeric(df_mobpro_brut['NPERR'], errors='coerce')
df_mobpro_brut['INPSM'] = pd.to_numeric(df_mobpro_brut['INPSM'], errors='coerce')
df_mobpro_brut['INPOM'] = pd.to_numeric(df_mobpro_brut['INPOM'], errors='coerce')
df_mobpro_brut['INEEM'] = pd.to_numeric(df_mobpro_brut['INEEM'], errors='coerce')

# Création des agrégations de base
mobpro_villes = df_mobpro_brut.groupby('COMMUNE').agg({
    'NPERR': 'sum',
    'INPSM': 'sum',
    'INPOM': 'sum',
    'INEEM': 'sum'})

# Créons d'abord un DataFrame temporaire pour chaque valeur d'ILT
for i in range(1, 8):
    # Grouper par COMMUNE et compter les occurrences où ILT == str(i)
    temp_count = df_mobpro_brut[df_mobpro_brut['ILT'] == i].groupby('COMMUNE').size()
    # Ajouter cette série au DataFrame principal
    mobpro_villes[f'ILT_{i}'] = temp_count

# Remplir les NaN par 0
mobpro_villes = mobpro_villes.fillna(0)

# Réinitialisation de l'index
mobpro_villes = mobpro_villes.reset_index()


In [ ]:
mobpro_villes

# Matrice des flux

On souhaite créer une matrice des flux pour avoir une vision générale des trajets domicile-travail quotidiens entre les villes. Pour chaque couple de ville, le nombre correspondant dans la matrice correspond au nombre de commute effectué de la ville 1 à la ville 2.

On remarque que pour la variable "COMMUNE" (commune de résidence) on a uniquement 75056 pour Paris (idem pour Lyon et Marseille), tandis que pour la variable "DCLT" (commune de travail) on a un découpage en arrondissement avec les codes 75101,... pour Paris (idem pour Lyon et Marseille). Afin de pallier cette différence, on se cale sur l'échelle des arrondissements en remplaçant la valeur de "COMMUNE" par celle de "ARM" lorsque la ville en question est Paris, Marseille ou Lyon.

In [ ]:
# On vérifie qu'il y a autant de lignes avec un arrondissement indiqué et de lignes dont la ville de résidence est soit Paris, soit Marseille, soit Lyon (cohérence entre "ARM" et "COMMUNE")
assert len(df_mobpro_brut[df_mobpro_brut['ARM']!="ZZZZZ"])== len(df_mobpro_brut[
    (df_mobpro_brut['COMMUNE'] == '75056') | 
    (df_mobpro_brut['COMMUNE'] == '13055') | 
    (df_mobpro_brut['COMMUNE'] == '69123'])

# Liste des codes insee d'arrondissements
arr_paris = [f"751{str(i).zfill(2)}" for i in range(1, 21)]
arr_marseille = [f"132{str(i).zfill(2)}" for i in range(1, 17)]
arr_lyon = [f"6938{str(i).zfill(1)}" for i in range(1, 10)]

In [66]:
df_mobpro_brut.loc[
    df_mobpro_brut['COMMUNE'].isin(['75056', '13055', '69123']), 'COMMUNE'
] = df_mobpro_brut['ARM']

On établit maintenant la matrice des flux.

In [ ]:
# Création d'une table croisée dynamique pour compter les déplacements
matrice_flux = pd.crosstab(df_mobpro_brut['COMMUNE'], df_mobpro_brut['DCLT'])

# Conversion en DataFrame pour plus de clarté
matrice_flux = pd.DataFrame(matrice_flux)

# En option, si vous voulez réinitialiser l'index pour avoir COMMUNE comme colonne
# matrice_flux = matrice_flux.reset_index()

# Pour voir les dimensions de la matrice
print("Dimensions de la matrice :", matrice_flux.shape)

In [ ]:
matrice_flux

In [15]:
def flux(ville_a,ville_b):
    '''
    Prends en arg deux codes insee en chaîne de caractère et renvoie le nombre de commute entre ces deux villes
    '''
    flux = matrice_flux.loc[ville_a, ville_b]
    print(f"Nombre de personnes se déplaçant de {ville_a} vers {ville_b} : {flux}")


In [ ]:
flux('78220', '75108')

In [ ]:
# Top 10 des villes générant le plus de déplacements
top_villes_depart = matrice_flux.sum(axis=1).sort_values(ascending=False).head(10)
top_villes_arrivee = matrice_flux.sum(axis=0).sort_values(ascending=False).head(10)

print("Top 10 villes de départ :\n", top_villes_depart)
print("Top 10 villes d'arrivée :\n", top_villes_arrivee)



In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Création du graphe
G = nx.Graph()

# Ajouter les arêtes (villes liées par des flux)
for index, row in matrice_flux.iterrows():
    for col, val in row.items():
        if val > 1000:  # Seulement si le flux est significatif
            G.add_edge(index, col, weight=val)

# Dessiner le graphe
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=700, font_size=10, edge_color='gray')

plt.title("Graphique des flux domicile-travail")
plt.show()


Idée : 

Identification des zones à forte dépendance à la voiture

Estimation des émissions de CO₂ par trajet (introduire la distance entre les villes)